In [0]:
import os
os.chdir(os.getcwd())

In [0]:
import pandas as pd
import numpy as np
import xlsxwriter
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

Coulomb_Amontons (CA) linear friction model 

$$F_f=\mu F_N$$

In [0]:
def func_CA(x, a):
    return a * x

Tabor-Bowden (TB) friction model 

$$F_f=F_a+F_d$$
$$F_a=\tau_s A_r$$

$$F_f=aF_N^n+cF_N$$
where is $n$ is either $1/2$ or $2/3$

In [0]:
def func_TB23(x, a, b):
    return a * x**(2/3.0) +b*x

In [0]:
def func_TB12(x, a, b):
    return a * x**(0.5) +b*x

In [0]:
surcons=('bs perpendicular','ss parallel parallel','ss parallel perpendicular','ss perpendicular perpendicular')
headers=('static_strain(mm)','static_force','min_mean','min_del','min_del_mad','max_mean','max_del','max_del_mad')
headers2=('mu_CA_s','mu_CA_k','tau0_TB_s','alpha_TB_s','tau0_TB_k','alpha_TB_k')
areas=[900,1700,2500]
# speeds=[50,100,150,200,300,400,600,800,1000,1200,1400,1600,1800]
Ec=1.32*10**9

In [0]:
# %matplotlib qt
surcon='ss perpendicular perpendicular'
writer = pd.ExcelWriter(surcon + '\\analysis_20181011.xlsx', engine='xlsxwriter')
for a in areas:
    file = surcon + '\\' + str(a) + 'mm\\analysis.xlsx'
    xls = pd.ExcelFile(file)
    speeds = [int(s) for s in xls.sheet_names]
    df_a=pd.DataFrame(columns=headers2,index=speeds)
    for s in speeds:
        df_sp = pd.read_excel(file, index_col=None, sheet_name=str(s))
        df_sp = df_sp.reset_index()
        df_sp['index'] = df_sp['index'].astype(int)
        df_sp = df_sp.set_index('index')
        popt1, pcov1 = curve_fit(func_CA, df_sp.index*9.81/1000, df_sp['static_force'].values.astype(float))
#         print(a,',',s,',',popt)
        df_a['mu_CA_s'][s]=popt1[0]
    
        popt2, pcov2 = curve_fit(func_CA, df_sp.index*9.81/1000, df_sp['max_mean'].values.astype(float))
        df_a['mu_CA_k'][s]=popt2[0]
        
#         popt3, pcov3 = curve_fit(func_TB12, df_sp.index*9.81/1000, df_sp['static_force'].values.astype(float),p0=[0.1,popt1[0]],bounds=(0,[np.inf, 1]))
        popt3, pcov3 = curve_fit(func_TB23, df_sp.index*9.81/1000, df_sp['static_force'].values.astype(float),p0=[0.1,popt1[0]],bounds=(0,[np.inf, 1]))
#         df_a['tau0_TB_s'][s]=popt3[0]*((Ec/(a/1000**2))**0.5)/2
        df_a['tau0_TB_s'][s]=popt3[0]*4*Ec*(2/3.0)/np.pi/(a/1000**2)**(1/3.0)
#         df_a['tau0_TB_s'][s]=popt3[0]
        df_a['alpha_TB_s'][s]=popt3[0]
    
#         popt4, pcov4 = curve_fit(func_TB12, df_sp.index*9.81/1000, df_sp['max_mean'].values.astype(float),p0=[0.1,popt2[0]],bounds=(0,[np.inf, 1]))
        popt4, pcov4 = curve_fit(func_TB23, df_sp.index*9.81/1000, df_sp['max_mean'].values.astype(float),p0=[0.1,popt2[0]],bounds=(0,[np.inf, 1]))
#         df_a['tau0_TB_k'][s]=popt4[0]*((Ec/(a/1000**2))**0.5)/2
        df_a['tau0_TB_k'][s]=popt4[0]*4*Ec*(2/3.0)/np.pi/(a/1000**2)**(1/3.0)
        df_a['alpha_TB_k'][s]=popt4[0]
        
#     plt.plot(speeds,df_a['mu_CA_k'], label=str(a))
    df_a.to_excel(writer, sheet_name=str(a))

# plt.legend()
# plt.show()
writer.save()